## Bibliotecas

In [17]:
# Import necessary modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager  # For automatic driver management
import time
import os
import traceback
import json
import requests

## Inicia Drive

In [18]:
def iniciar_driver():
    # Initialize the WebDriver for Microsoft Edge
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service)
    return driver

# Start the driver
driver = iniciar_driver()


## Navega para o login

In [19]:
# Navigate to the login page
url_login = 'https://pajucarahoteis.hitspms.net/#/login'
driver.get(url_login)

# Optionally, wait for the page to load completely
time.sleep(5)  # Adjust the sleep time as necessary


In [20]:
try:
    # Wait for the email field to be present
    email_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'Email'))
    )
    print("Email field found.")
    
    # Wait for the password field to be present
    password_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'Password'))
    )
    print("Password field found.")
except Exception as e:
    print(f"An error occurred while locating input fields: {type(e).__name__}: {e}")
    traceback.print_exc()
    driver.quit()


Email field found.
Password field found.


In [21]:
# Use environment variables for credentials
email = os.environ.get('LOGIN_EMAIL_HITS')
senha = os.environ.get('LOGIN_SENHA_HITS')

# Ensure that the credentials are set
if not email or not senha:
    print("Please set the environment variables LOGIN_EMAIL and LOGIN_SENHA.")
    driver.quit()
else:
    # Enter the credentials
    email_field.send_keys(email)
    password_field.send_keys(senha)
    print("Credentials entered.")


Credentials entered.


## Clica para fazer login

In [22]:
try:
    # Locate the login button
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.button-navbar.hover-tag[type="submit"]'))
    )
    print("Login button found.")
    
    # Click the login button
    login_button.click()
    print("Login button clicked.")
    time.sleep(5)
except Exception as e:
    print(f"An error occurred while clicking the login button: {type(e).__name__}: {e}")
    traceback.print_exc()
    driver.quit()


Login button found.
Login button clicked.


## Acessa registro

In [23]:
def get_bearer_token(driver):
    try:
        # Retrieve the token from localStorage
        token = driver.execute_script("return localStorage.getItem('authorizationDataToken');")
        if token:
            print("Bearer token retrieved from localStorage.")
            return token
        else:
            print("Bearer token not found in localStorage.")
            return None
    except Exception as e:
        print(f"An error occurred while retrieving the token: {type(e).__name__}: {e}")
        traceback.print_exc()
        return None
bearer_token = get_bearer_token(driver)

Bearer token retrieved from localStorage.


## Tenta decodificar o token

In [24]:
import jwt

# After decoding the token to check expiration
try:
    token_payload = jwt.decode(bearer_token, options={"verify_signature": False})
    print("Token Payload:", token_payload)
    
    # Extract 'nbf' and 'exp' from the token payload
    nbf = token_payload.get('nbf')
    exp = token_payload.get('exp')
    
    # Import datetime module
    from datetime import datetime

    # Convert 'nbf' and 'exp' to datetime objects and print them
    if nbf:
        nbf_date = datetime.utcfromtimestamp(nbf)
        print(f"Token valid from (nbf): {nbf_date}")
    else:
        print("Not Before (nbf) claim not found in token.")

    if exp:
        exp_date = datetime.utcfromtimestamp(exp)
        print(f"Token expires at (exp): {exp_date}")
    else:
        print("Expiration time (exp) not found in token.")

    # Get current UTC time
    current_time = datetime.utcnow()
    print(f"Current UTC time: {current_time}")

    # Check if the token is valid
    if nbf and current_time < nbf_date:
        print("Token is not yet valid.")
        driver.quit()
        exit(1)
    elif exp and current_time > exp_date:
        print("Token has expired.")
        driver.quit()
        exit(1)
    else:
        print("Token is currently valid.")
except Exception as e:
    print(f"Error decoding token: {e}")

Token Payload: {'nbf': 1730944904, 'exp': 1731031304, 'iss': 'https://susceptor.apphotel.one', 'aud': ['https://susceptor.apphotel.one/resources', 'webapi'], 'client_id': 'B37748FC-ED13-4858-AE26-28AB3512A171', 'sub': '1c8f1b3e-57dd-40ee-9d96-476e8690c29b', 'auth_time': 1730944904, 'idp': 'local', 'scope': ['openid', 'profile', 'webapi'], 'amr': ['pwd']}
Token valid from (nbf): 2024-11-07 02:01:44
Token expires at (exp): 2024-11-08 02:01:44
Current UTC time: 2024-11-07 02:01:55.099014
Token is currently valid.


## Define o caminho para salvar o JSON:

In [25]:
# Define the directory and file name
directory = r'C:\Users\Weslley Carvalho\OneDrive\Weslley & Beatriz - Arquivos\Trabalho\Projetos\Weslley e Fellype\Grupo Pajuçara Hotel\Scripts\Dados'
file_name = 'api_response.json'
file_path = os.path.join(directory, file_name)

## Requisita os dados do relatório

In [26]:
# Define the API endpoint
api_url = 'https://pajucarahoteis.hitspms.net/api/DailiesPosted/GetReportAsync'

# Define the query parameters
params = {
    'DateCriteria': '1',
    'StartDate': '2024-11-01',
    'EndDate': '2024-11-06'
}

# Define the headers
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Authorization': f'Bearer {bearer_token}',
    'Connection': 'keep-alive',
    'Referer': 'https://pajucarahoteis.hitspms.net/',
    'Sec-CH-UA': '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0',
    'X-API-Application-Code': '1',
    'X-API-Language-Code': 'pt-br',
    'X-API-Property-Code': '2',
    'X-API-Version': '2024.6.93',
}

# Get cookies from Selenium
selenium_cookies = driver.get_cookies()

# Create a requests session
session = requests.Session()

# Add cookies to the session
for cookie in selenium_cookies:
    # Convert Selenium cookie dict to requests cookie dict
    session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])


# Create a custom session and adapter to disable the 'Expect' header
from requests.adapters import HTTPAdapter

class CustomHTTPAdapter(HTTPAdapter):
    def add_headers(self, request, **kwargs):
        # Remove the 'Expect' header if it exists
        request.headers.pop('Expect', None)

# Use the custom adapter in your session
session = requests.Session()
session.mount('https://', CustomHTTPAdapter())
session.headers.update(headers)

# Make the GET request without the 'Expect' header
response = session.get(api_url, params=params)

time.sleep(5)

# Check the response status code
if response.status_code == 200:
    print("API request successful.")
    # Process the JSON data
    data = response.json()
    
    # Save the data to a JSON file
    try:
        # Ensure the directory exists
        os.makedirs(directory, exist_ok=True)
        
        # Write the data to the file
        with open(file_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f"Data saved to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving the data: {type(e).__name__}: {e}")
else:
    print(f"API request failed with status code {response.status_code}: {response.text}")


API request successful.
Data saved to C:\Users\Weslley Carvalho\OneDrive\Weslley & Beatriz - Arquivos\Trabalho\Projetos\Weslley e Fellype\Grupo Pajuçara Hotel\Scripts\Dados\api_response.json


In [27]:
# Close the driver
# driver.quit()
